In [8]:
from datasets import load_dataset
import pandas as pd

# Step 3: Load the full phishing dataset from Hugging Face
dataset = load_dataset("renemel/compiled-phishing-dataset", split="train")

# Step 4: Convert to pandas DataFrame
df = dataset.to_pandas()

# Step 5: Preview the column names to identify date column
print("Columns available:")
print(df.columns)

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 119148/119148 [00:00<00:00, 555250.80 examples/s]

Columns available:
Index(['text', 'type', 'source', 'word_count', 'sentence_count',
       'words_per_sentence', 'domain'],
      dtype='object')
